## DEBUG DATA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy import signal

# ============================================================================
# CONFIGURATION
# ============================================================================
FILENAME = "debug-data/raw_data_ch19_20251015_045125.bin"
TARGET_INDEX = 2297855
SAMPLING_RATE = 30000  # Hz
WINDOW_SECONDS = 0.1  # Show ±0.1 seconds around the target index
CONTEXT_SECONDS = 1.0  # Show a wider context view as well

# Filter parameters
LOW_FREQ = 0.25  # Hz
HIGH_FREQ = 4.0  # Hz
FILTER_ORDER = 4  # Butterworth filter order

# ============================================================================
# Read Raw Data
# ============================================================================
if not os.path.exists(FILENAME):
    print(f"Error: File '{FILENAME}' not found")
else:
    # Read data
    data = np.fromfile(FILENAME, dtype=np.float64)
    print(f"File: {FILENAME}")
    print(f"Total samples: {len(data):,}")
    print(f"Target index: {TARGET_INDEX:,}")
    
    # ============================================================================
    # Apply Bandpass Filter (0.25-4 Hz)
    # ============================================================================
    print(f"\nApplying {LOW_FREQ}-{HIGH_FREQ} Hz bandpass filter...")
    
    # Design Butterworth bandpass filter
    nyquist = SAMPLING_RATE / 2
    low_norm = LOW_FREQ / nyquist
    high_norm = HIGH_FREQ / nyquist
    
    # Create stable filter using second-order sections (SOS)
    sos = signal.butter(FILTER_ORDER, [low_norm, high_norm], btype='band', output='sos')
    
    # Apply zero-phase filtering using sosfiltfilt (prevents NaNs)
    filtered_data = signal.sosfiltfilt(sos, data)

    
    print(f"Filter applied successfully")
    print(f"Original data range: [{np.min(data):.2f}, {np.max(data):.2f}] µV")
    print(f"Filtered data range: [{np.min(filtered_data):.2f}, {np.max(filtered_data):.2f}] µV")
    
    # Check if index is valid
    if TARGET_INDEX < 0 or TARGET_INDEX >= len(data):
        print(f"Error: Target index {TARGET_INDEX} is out of range (0 to {len(data)-1})")
    else:
        # Calculate window indices
        window_samples = int(WINDOW_SECONDS * SAMPLING_RATE)
        context_samples = int(CONTEXT_SECONDS * SAMPLING_RATE)
        
        # Close-up view indices
        start_idx = max(0, TARGET_INDEX - window_samples)
        end_idx = min(len(data), TARGET_INDEX + window_samples)
        
        # Context view indices
        context_start = max(0, TARGET_INDEX - context_samples)
        context_end = min(len(data), TARGET_INDEX + context_samples)
        
        # Create figure with four subplots (2x2)
        fig = plt.figure(figsize=(18, 12))
        
        # ============================================================================
        # Plot 1: Original Context View (±1 second)
        # ============================================================================
        ax1 = plt.subplot(2, 2, 1)
        context_indices = np.arange(context_start, context_end)
        context_time = context_indices / SAMPLING_RATE
        
        ax1.plot(context_time, data[context_start:context_end], 'b-', linewidth=0.5, alpha=0.7)
        ax1.axvline(x=TARGET_INDEX/SAMPLING_RATE, color='red', linestyle='--', linewidth=2, label=f'Target Index: {TARGET_INDEX}')
        
        # Highlight the zoomed region
        zoom_start_time = start_idx / SAMPLING_RATE
        zoom_end_time = end_idx / SAMPLING_RATE
        ax1.axvspan(zoom_start_time, zoom_end_time, alpha=0.2, color='yellow', label='Zoomed Region')
        
        ax1.set_xlabel('Time (seconds)', fontsize=12)
        ax1.set_ylabel('Amplitude (µV)', fontsize=12)
        ax1.set_title(f'Original Signal - Context View (±{CONTEXT_SECONDS}s)', fontsize=14)
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        
        # ============================================================================
        # Plot 2: Filtered Context View (±1 second)
        # ============================================================================
        ax2 = plt.subplot(2, 2, 2)
        ax2.plot(context_time, filtered_data[context_start:context_end], 'g-', linewidth=0.5, alpha=0.7)
        ax2.axvline(x=TARGET_INDEX/SAMPLING_RATE, color='red', linestyle='--', linewidth=2, label=f'Target Index: {TARGET_INDEX}')
        ax2.axvspan(zoom_start_time, zoom_end_time, alpha=0.2, color='yellow', label='Zoomed Region')
        
        ax2.set_xlabel('Time (seconds)', fontsize=12)
        ax2.set_ylabel('Amplitude (µV)', fontsize=12)
        ax2.set_title(f'Filtered Signal ({LOW_FREQ}-{HIGH_FREQ} Hz) - Context View', fontsize=14)
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        
        # ============================================================================
        # Plot 3: Original Close-up View (±0.1 second)
        # ============================================================================
        ax3 = plt.subplot(2, 2, 3)
        close_indices = np.arange(start_idx, end_idx)
        close_time = close_indices / SAMPLING_RATE
        
        ax3.plot(close_time, data[start_idx:end_idx], 'b-', linewidth=1.0)
        ax3.plot(TARGET_INDEX/SAMPLING_RATE, data[TARGET_INDEX], 'ro', markersize=10, 
                 label=f'Index {TARGET_INDEX}: {data[TARGET_INDEX]:.2f} µV')
        
        ax3.set_xlabel('Time (seconds)', fontsize=12)
        ax3.set_ylabel('Amplitude (µV)', fontsize=12)
        ax3.set_title(f'Original Signal - Close-up View (±{WINDOW_SECONDS}s)', fontsize=14)
        ax3.grid(True, alpha=0.3)
        ax3.legend()
        
        # ============================================================================
        # Plot 4: Filtered Close-up View (±0.1 second)
        # ============================================================================
        ax4 = plt.subplot(2, 2, 4)
        ax4.plot(close_time, filtered_data[start_idx:end_idx], 'g-', linewidth=1.0)
        ax4.plot(TARGET_INDEX/SAMPLING_RATE, filtered_data[TARGET_INDEX], 'ro', markersize=10, 
                 label=f'Index {TARGET_INDEX}: {filtered_data[TARGET_INDEX]:.2f} µV')
        
        # Add some neighboring points
        for i in range(-5, 6):
            if 0 <= TARGET_INDEX + i < len(filtered_data):
                ax4.plot((TARGET_INDEX + i)/SAMPLING_RATE, filtered_data[TARGET_INDEX + i], 'o', 
                        color='orange', markersize=4, alpha=0.6)
        
        ax4.set_xlabel('Time (seconds)', fontsize=12)
        ax4.set_ylabel('Amplitude (µV)', fontsize=12)
        ax4.set_title(f'Filtered Signal ({LOW_FREQ}-{HIGH_FREQ} Hz) - Close-up View', fontsize=14)
        ax4.grid(True, alpha=0.3)
        ax4.legend()
        
        # ============================================================================
        # Print detailed information
        # ============================================================================
        print("\nSignal Information at Target Index:")
        print(f"  Time: {TARGET_INDEX/SAMPLING_RATE:.6f} seconds")
        print(f"  Original value: {data[TARGET_INDEX]:.2f} µV")
        print(f"  Filtered value: {filtered_data[TARGET_INDEX]:.2f} µV")
        
        print(f"\nSurrounding filtered values (±5 samples):")
        for i in range(-5, 6):
            if 0 <= TARGET_INDEX + i < len(filtered_data):
                print(f"  Index {TARGET_INDEX + i}: {filtered_data[TARGET_INDEX + i]:.2f} µV")
        
        # Calculate local statistics
        local_window = 100  # samples for local stats
        local_start = max(0, TARGET_INDEX - local_window)
        local_end = min(len(data), TARGET_INDEX + local_window)
        local_data_orig = data[local_start:local_end]
        local_data_filt = filtered_data[local_start:local_end]
        
        print(f"\nLocal statistics (±{local_window} samples):")
        print("  Original signal:")
        print(f"    Mean: {np.mean(local_data_orig):.2f} µV")
        print(f"    Std: {np.std(local_data_orig):.2f} µV")
        print(f"    Min: {np.min(local_data_orig):.2f} µV")
        print(f"    Max: {np.max(local_data_orig):.2f} µV")
        print("  Filtered signal:")
        print(f"    Mean: {np.mean(local_data_filt):.2f} µV")
        print(f"    Std: {np.std(local_data_filt):.2f} µV")
        print(f"    Min: {np.min(local_data_filt):.2f} µV")
        print(f"    Max: {np.max(local_data_filt):.2f} µV")
        
        # Adjust layout and show
        plt.tight_layout()
        plt.savefig('signal_at_index_2297855_filtered.png', dpi=150, bbox_inches='tight')
        print(f"\nFigure saved as: signal_at_index_2297855_filtered.png")
        plt.show()

## convert/output to matlab

In [ ]:
import numpy as np
import os
import csv

# ============================================================================
# CONFIGURATION
# ============================================================================
INPUT_FILENAME  = "debug-data/raw_data_ch19_20251015_045125.bin"
OUTPUT_FILENAME = "raw_data_ch19_20251015_045125.csv"

# ============================================================================
# Read raw binary and export to CSV
# ============================================================================
if not os.path.exists(INPUT_FILENAME):
    print(f"Error: File '{INPUT_FILENAME}' not found")
else:
    print(f"Reading binary data from: {INPUT_FILENAME}")

    # Read float64 data from .bin file
    data = np.fromfile(INPUT_FILENAME, dtype=np.float64)

    print(f"Samples loaded: {len(data):,}")
    print(f"Writing CSV file to: {OUTPUT_FILENAME}")

    # Write to CSV (one sample per line)
    with open(OUTPUT_FILENAME, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["sample_index", "value"])  # header

        for idx, val in enumerate(data):
            writer.writerow([idx, val])

    print("\n✅ Export complete!")
    print(f"CSV saved as: {OUTPUT_FILENAME}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from datetime import datetime

# ============================================================================
# CONFIGURATION
# ============================================================================
DATA_FOLDER = "debug-data"      # Folder containing .bin files
SAMPLING_RATE = 30000           # Hz
DURATION_TO_PLOT = 10.0         # seconds - show first 10 seconds

# ============================================================================
# Find all .bin files
# ============================================================================
bin_files = sorted(glob.glob(os.path.join(DATA_FOLDER, "*.bin")))

if not bin_files:
    print(f"Error: No .bin files found in '{DATA_FOLDER}'")

else:
    print(f"Found {len(bin_files)} .bin file(s) in '{DATA_FOLDER}'")
    print("=" * 80)

    n_files = len(bin_files)
    fig, axes = plt.subplots(n_files, 1, figsize=(16, 3 * n_files), sharex=True)

    if n_files == 1:
        axes = [axes]

    # ---------------------------------------------------------
    # PROCESS + NORMALIZE + PLOT
    # ---------------------------------------------------------
    for file_idx, (filename, ax) in enumerate(zip(bin_files, axes)):
        print(f"\nProcessing file {file_idx + 1}/{n_files}: {os.path.basename(filename)}")

        file_size = os.path.getsize(filename)
        expected_samples = file_size // 8
        duration_sec = expected_samples / SAMPLING_RATE

        try:
            data = np.fromfile(filename, dtype=np.float64)

            # Compute statistics BEFORE normalization
            mean_val = np.mean(data)
            std_val = np.std(data)
            min_val = np.min(data)
            max_val = np.max(data)

            # ---------------------------
            # ✅ Normalize (Z–score)
            # ---------------------------
            data_norm = (data - mean_val) / std_val

            # Prepare for plotting
            samples_to_plot = min(int(DURATION_TO_PLOT * SAMPLING_RATE), len(data_norm))
            time = np.arange(samples_to_plot) / SAMPLING_RATE

            # Plot normalized signal
            ax.plot(time, data_norm[:samples_to_plot], 'b-', linewidth=0.5)
            ax.set_ylabel("Z-score", fontsize=10)
            ax.grid(True, alpha=0.3)

            # Info text for subplot title
            ax.set_title(
                f"{os.path.basename(filename)}   "
                f"(raw mean={mean_val:.2f} µV, std={std_val:.2f} µV, "
                f"min={min_val:.1f}, max={max_val:.1f})",
                fontsize=9, loc='left'
            )

            print(f"  Processed: {len(data):,} samples ({duration_sec:.1f} seconds)")
            print(f"  Raw signal stats → Mean={mean_val:.2f} µV, Std={std_val:.2f} µV")

        except Exception as e:
            print(f"  Error reading {filename}: {e}")
            ax.text(0.5, 0.5, f"Error reading file", ha='center', va='center', transform=ax.transAxes)

    # ---------------------------------------------------------
    # ✅ ENFORCE SAME SCALE FOR ALL SUBPLOTS
    # ---------------------------------------------------------
    for ax in axes:
        ax.set_ylim(-3, 3)      # 3 SD covers almost all neural events

    axes[-1].set_xlabel("Time (seconds)", fontsize=12)

    fig.suptitle(f"Normalized Neural Recordings Summary - {DATA_FOLDER}", fontsize=14, y=0.995)
    plt.tight_layout()

    output_filename = "neural_recordings_summary_normalized.png"
    plt.savefig(output_filename, dpi=150, bbox_inches='tight')
    print(f"\nSummary figure saved as: {output_filename}")

    plt.show()

    print("\n" + "=" * 80)
    print(f"Completed processing {n_files} file(s)")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from scipy import signal
import glob

# ============================================================================
# CONFIGURATION
# ============================================================================
DATA_FOLDER = "debug-data"  # Folder containing .bin files
SAMPLING_RATE = 30000  # Hz
DURATION_TO_PLOT = 10.0  # seconds

# ============================================================================
# Find all .bin files
# ============================================================================
bin_files = glob.glob(os.path.join(DATA_FOLDER, "*.bin"))

if not bin_files:
    print(f"Error: No .bin files found in '{DATA_FOLDER}'")
else:
    print(f"Found {len(bin_files)} .bin file(s) in '{DATA_FOLDER}'")
    print("=" * 80)
    
    # Process each file
    for file_idx, filename in enumerate(bin_files):
        print(f"\nProcessing file {file_idx + 1}/{len(bin_files)}: {os.path.basename(filename)}")
        print("-" * 80)
        
        # ============================================================================
        # Read Raw Data
        # ============================================================================
        # Get file information
        file_size = os.path.getsize(filename)
        expected_samples = file_size // 8  # 8 bytes per double
        
        print(f"File: {filename}")
        print(f"Size: {file_size:,} bytes")
        print(f"Expected samples: {expected_samples:,}")
        print(f"Expected duration: {expected_samples / SAMPLING_RATE:.2f} seconds")
        print()
        
        # Read as flat array of doubles
        data = np.fromfile(filename, dtype=np.float64)
        print(f"Successfully read {len(data):,} samples")
        print(f"Data type: {data.dtype}")
        print(f"Shape: {data.shape}")
        print()
        
        # ============================================================================
        # Data Statistics
        # ============================================================================
        print("Data Statistics:")
        print(f"  Mean: {np.mean(data):.2f} µV")
        print(f"  Std Dev: {np.std(data):.2f} µV")
        print(f"  Min: {np.min(data):.2f} µV")
        print(f"  Max: {np.max(data):.2f} µV")
        print()
        
        # Check for issues
        if np.any(np.isnan(data)):
            print("WARNING: Data contains NaN values!")
        if np.any(np.isinf(data)):
            print("WARNING: Data contains Inf values!")
        
        # Show first few samples
        print("First 20 samples (µV):")
        print(data[:20])
        
        # ============================================================================
        # Plot Data
        # ============================================================================
        samples_to_plot = int(DURATION_TO_PLOT * SAMPLING_RATE)
        samples_to_plot = min(samples_to_plot, len(data))
        time = np.arange(samples_to_plot) / SAMPLING_RATE
        
        plt.figure(figsize=(14, 6))
        plt.plot(time, data[:samples_to_plot], 'b-', linewidth=0.5)
        plt.xlabel('Time (seconds)', fontsize=12)
        plt.ylabel('Amplitude (µV)', fontsize=12)
        plt.title(f'Neural Signal - {os.path.basename(filename)} - First {DURATION_TO_PLOT} seconds', fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # ============================================================================
        # Plot Power Spectral Density
        # ============================================================================
        # Use first 10 seconds of data for PSD
        psd_duration = min(10.0, len(data) / SAMPLING_RATE)
        psd_samples = int(psd_duration * SAMPLING_RATE)
        
        freqs, psd = signal.welch(data[:psd_samples], 
                                  fs=SAMPLING_RATE, 
                                  nperseg=4096)
        
        plt.figure(figsize=(14, 6))
        plt.semilogy(freqs, psd)
        plt.xlabel('Frequency (Hz)', fontsize=12)
        plt.ylabel('Power Spectral Density (µV²/Hz)', fontsize=12)
        plt.title(f'Power Spectral Density - {os.path.basename(filename)}', fontsize=14)
        plt.xlim([0, 500])  # Focus on 0-500 Hz range
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # ============================================================================
        # Optional: Save as CSV (with downsampling for large files)
        # ============================================================================
        # Uncomment to save as CSV:
        # DOWNSAMPLE_FACTOR = 10  # Reduce file size by this factor
        # data_downsampled = data[::DOWNSAMPLE_FACTOR]
        # time_downsampled = np.arange(len(data_downsampled)) * DOWNSAMPLE_FACTOR / SAMPLING_RATE
        # 
        # df = pd.DataFrame({
        #     'time_sec': time_downsampled,
        #     'amplitude_uV': data_downsampled
        # })
        # 
        # output_filename = filename.replace('.bin', '_downsampled.csv')
        # df.to_csv(output_filename, index=False)
        # print(f"Saved to: {output_filename}")
        # print(f"New sampling rate: {SAMPLING_RATE/DOWNSAMPLE_FACTOR:.0f} Hz")
        
    print("\n" + "=" * 80)
    print(f"Completed processing {len(bin_files)} file(s)")

## GOOD DATA REFERENCE

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# ============================================================================
# CONFIGURATION
# ============================================================================
FILENAME = "raw_binary_data/raw_data_ch1_20251006_143204.bin"  # Change this to your file
SAMPLING_RATE = 30000  # Hz
DURATION_TO_PLOT = 1.0  # seconds

# ============================================================================
# Read Raw Data
# ============================================================================

if not os.path.exists(FILENAME):
    print(f"Error: File '{FILENAME}' not found")
else:
    # Get file information
    file_size = os.path.getsize(FILENAME)
    expected_samples = file_size // 8  # 8 bytes per double
    
    print(f"File: {FILENAME}")
    print(f"Size: {file_size:,} bytes")
    print(f"Expected samples: {expected_samples:,}")
    print(f"Expected duration: {expected_samples / SAMPLING_RATE:.2f} seconds")
    print()
    
    # Read as flat array of doubles
    data = np.fromfile(FILENAME, dtype=np.float64)
    print(f"Successfully read {len(data):,} samples")
    print(f"Data type: {data.dtype}")
    print(f"Shape: {data.shape}")
    print()

# ============================================================================
# Data Statistics
# ============================================================================

print("Data Statistics:")
print(f"  Mean: {np.mean(data):.2f} µV")
print(f"  Std Dev: {np.std(data):.2f} µV")
print(f"  Min: {np.min(data):.2f} µV")
print(f"  Max: {np.max(data):.2f} µV")
print()

# Check for issues
if np.any(np.isnan(data)):
    print("WARNING: Data contains NaN values!")
if np.any(np.isinf(data)):
    print("WARNING: Data contains Inf values!")

# Show first few samples
print("First 20 samples (µV):")
print(data[:20])

# ============================================================================
# Plot Data
# ============================================================================

samples_to_plot = int(DURATION_TO_PLOT * SAMPLING_RATE)
samples_to_plot = min(samples_to_plot, len(data))

time = np.arange(samples_to_plot) / SAMPLING_RATE

plt.figure(figsize=(14, 6))
plt.plot(time, data[:samples_to_plot], 'b-', linewidth=0.5)
plt.xlabel('Time (seconds)', fontsize=12)
plt.ylabel('Amplitude (µV)', fontsize=12)
plt.title(f'Neural Signal - First {DURATION_TO_PLOT} seconds', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ============================================================================
# Plot Power Spectral Density
# ============================================================================

from scipy import signal

# Use first 10 seconds of data for PSD
psd_duration = min(10.0, len(data) / SAMPLING_RATE)
psd_samples = int(psd_duration * SAMPLING_RATE)

freqs, psd = signal.welch(data[:psd_samples], 
                          fs=SAMPLING_RATE, 
                          nperseg=4096)

plt.figure(figsize=(14, 6))
plt.semilogy(freqs, psd)
plt.xlabel('Frequency (Hz)', fontsize=12)
plt.ylabel('Power Spectral Density (µV²/Hz)', fontsize=12)
plt.title('Power Spectral Density', fontsize=14)
plt.xlim([0, 500])  # Focus on 0-500 Hz range
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ============================================================================
# Optional: Save as CSV (with downsampling for large files)
# ============================================================================

# Uncomment to save as CSV:
# DOWNSAMPLE_FACTOR = 10  # Reduce file size by this factor
# data_downsampled = data[::DOWNSAMPLE_FACTOR]
# time_downsampled = np.arange(len(data_downsampled)) * DOWNSAMPLE_FACTOR / SAMPLING_RATE
# 
# df = pd.DataFrame({
#     'time_sec': time_downsampled,
#     'amplitude_uV': data_downsampled
# })
# 
# output_filename = FILENAME.replace('.bin', '_downsampled.csv')
# df.to_csv(output_filename, index=False)
# print(f"Saved to: {output_filename}")
# print(f"New sampling rate: {SAMPLING_RATE/DOWNSAMPLE_FACTOR:.0f} Hz")